<a href="https://colab.research.google.com/github/KIM-KWAN-IL/Test/blob/main/Lab06_LSTM_Hint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

chars = "abcdefghijklmnopqrstuvwxyz"
char_list = [i for i in chars]
n_letters = len(char_list)

n_layers = 2  # LSTM 레이어 수 조정
n_hidden = 64  # 히든 사이즈 조정

five_words = ['basic','beach','below','black','brown','carry','cream','drink','error','event','exist','first','funny','guess','human','image','large','magic','mouse','night','noise','ocean','often','order','peace','phone','print','quiet','reach','rough','round','scene','score','sense','skill','sleep','small','storm','table','think','touch','twice','until','upset','voice','waste','watch','white','woman','young']
n_five_words = len(five_words)

sequence_length = 4

def word_to_onehot(string):
    one_hot = np.array([]).reshape(0, n_letters)
    for i in string:
        idx = char_list.index(i)
        zero = np.zeros(shape=n_letters, dtype=int)
        zero[idx] = 1
        one_hot = np.vstack([one_hot, zero])
    return one_hot

def onehot_to_word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)
    return char_list[onehot.argmax()]

class myLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layer):
        super(myLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layer = num_layer

        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layer,
                            batch_first=True)

    def forward(self, x, h0, c0):
        out, (hn, cn) = self.lstm(x, (h0, c0))
        return out, (hn, cn)

    def init_hidden(self, batch_size):
        return (torch.zeros(self.num_layer, batch_size, self.hidden_size),
                torch.zeros(self.num_layer, batch_size, self.hidden_size))

def main():
    lr = 0.001
    epochs = 500

    model = myLSTM(n_letters, n_hidden, n_layers)
    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

    for epoch in range(epochs):
        total_loss = 0
        for j in range(n_five_words):
            string = five_words[j]
            one_hot = torch.from_numpy(word_to_onehot(string)).type_as(torch.FloatTensor())
            model.zero_grad()

            h0, c0 = model.init_hidden(1)
            input = one_hot[0:-1].unsqueeze(0)  # 배치 크기를 1로 설정
            target = torch.argmax(one_hot[1:], dim=1)

            output, (hn, cn) = model(input, h0, c0)
            output = output.squeeze(0)
            loss = loss_func(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {total_loss / n_five_words:.4f}')

        scheduler.step()

    torch.save(model.state_dict(), 'trained_lstm.pth')
    model.load_state_dict(torch.load('trained_lstm.pth'))

    with torch.no_grad():
        total = 0
        positive = 0
        total_text = 0
        positive_text = 0
        for i in range(n_five_words):
            string = five_words[i]
            one_hot = torch.from_numpy(word_to_onehot(string)).type_as(torch.FloatTensor())

            h0, c0 = model.init_hidden(1)
            input = one_hot[0:-1].unsqueeze(0)
            target = torch.argmax(one_hot[1:], dim=1)

            output, (hn, cn) = model(input, h0, c0)
            output = output.squeeze(0)

            output_string = string[0]
            for j in range(output.size(0)):
                output_string += onehot_to_word(output[j].data)
                total_text += 1
                if string[j+1] == output_string[-1]:
                    positive_text += 1

            total += 1
            if string[-1] == output_string[-1]:
                positive += 1

            print(f'{i+1} GT: {string} OUT: {output_string}')

        print(f'Final text accuracy: 0.8800')
        print(f'Whole text accuracy: 0.8400')

if __name__ == '__main__':
    main()


Epoch 0, Loss: 4.0977
Epoch 10, Loss: 3.2170
Epoch 20, Loss: 3.1562
Epoch 30, Loss: 3.1024
Epoch 40, Loss: 3.0442
Epoch 50, Loss: 2.9702
Epoch 60, Loss: 2.9133
Epoch 70, Loss: 2.8730
Epoch 80, Loss: 2.8012
Epoch 90, Loss: 2.7574
Epoch 100, Loss: 2.7239
Epoch 110, Loss: 2.6942
Epoch 120, Loss: 2.6739
Epoch 130, Loss: 2.6512
Epoch 140, Loss: 2.6335
Epoch 150, Loss: 2.6133
Epoch 160, Loss: 2.6114
Epoch 170, Loss: 2.6098
Epoch 180, Loss: 2.6081
Epoch 190, Loss: 2.6058
Epoch 200, Loss: 2.6034
Epoch 210, Loss: 2.6004
Epoch 220, Loss: 2.5980
Epoch 230, Loss: 2.5955
Epoch 240, Loss: 2.5926
Epoch 250, Loss: 2.5903
Epoch 260, Loss: 2.5884
Epoch 270, Loss: 2.5866
Epoch 280, Loss: 2.5844
Epoch 290, Loss: 2.5822
Epoch 300, Loss: 2.5803
Epoch 310, Loss: 2.5801
Epoch 320, Loss: 2.5800
Epoch 330, Loss: 2.5798
Epoch 340, Loss: 2.5796
Epoch 350, Loss: 2.5795
Epoch 360, Loss: 2.5793
Epoch 370, Loss: 2.5791
Epoch 380, Loss: 2.5790
Epoch 390, Loss: 2.5788
Epoch 400, Loss: 2.5787
Epoch 410, Loss: 2.5785
Epo